In [6]:
import pandas as pd 
import numpy as np 
import joblib 
import cv2 
from ultralytics import YOLO 
import torch 
from skimage.feature import hog



class OCRModel:
    def __init__(self):
        self.model1 = YOLO("yolov8n.pt")
        self.model1.model = torch.load("/home/jik19004/FilesToRun/DINGRESEARCH/CSE5717-Project/OCR/model/yolo_char_rec.pt")
        self.model2 = joblib.load("/home/jik19004/FilesToRun/DINGRESEARCH/CSE5717-Project/OCR/model/lreg_char_class.pkl")
        
    def prepare_img(self, img, xmin: float, ymin: float, xmax: float, ymax: float):
        # crop image to bounding box
        char_img = img[int(ymin):int(ymax), int(xmin):int(xmax)]

        # scale image
        char_img = cv2.resize(char_img, (64, 64))
        #plt.figure()
        hog_img = hog(char_img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(1, 1))

        return hog_img
    
        
    def predict(self, image_path):
        results = self.model1(image_path)
        pred = []
        
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        for result in results[0].boxes.xyxy:
            xmin, ymin, xmax, ymax = result[0:4]
            char_img = self.prepare_img(img, xmin, ymin, xmax, ymax)
            pred.append(self.model2.predict([char_img]))
        return pred

In [7]:
df = pd.read_csv("/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/parsedLP_Plates.csv")
df.drop("Unnamed: 0", axis = 1, inplace= True)
display(df)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,MH15TC554,0000.xml,M,13-17-27-32,H,25-16-38-33,1,36-16-46-34,5,45-16-55-33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MH02CB4545,0001.xml,M,15-23-32-41,H,33-26-46-44,0,51-29-64-47,2,63-32-77-50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KA031351,0003.xml,K,19-11-30-29,A,31-13-40-29,0,40-13-52-30,3,51-15-62-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TN0600000,0004.xml,T,156-27-228-156,N,248-31-322-159,0,362-32-435-159,6,456-38-526-161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KA51MJ8156,0005.xml,K,22-13-33-34,A,34-13-45-34,5,45-12-58-34,1,59-14-69-34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,MH2OBN3525,0232.xml,M,25-25-40-45,H,42-25-56-45,2,59-25-71-45,O,71-25-88-45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,MH01AR5274,0233.xml,M,21-19-31-29,H,30-19-39-28,0,43-20-51-28,1,50-19-57-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,MH20CS1941,0234.xml,M,18-18-31-35,H,32-18-42-35,2,43-19-54-34,0,54-18-64-34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,H20CS1938,0235.xml,H,21-15-27-26,2,27-16-32-26,0,32-17-37-27,C,37-16-43-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
TrainingPartition = int(len(df) * 0.4) - 1 
ValidationPartition = int(len(df) * 0.7) - 1 
print("index of Training Partition is:", TrainingPartition) 
print("index of Valiadtion Partition is:", ValidationPartition) 

index of Training Partition is: 82
index of Valiadtion Partition is: 144


In [9]:
from torchmetrics.text import CharErrorRate
import os 

ImageFiles = [os.path.join('/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images', file)
                   for file in os.listdir('/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images')] # get the file content. TestingDataList = ImageFiles[144:]
ImageFiles = sorted(ImageFiles)
TestingDataList = ImageFiles[144:]
print(TestingDataList)

['/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0163.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0164.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0167.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0168.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0169.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0170.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0171.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0172.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0173.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0174.png', '/home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/image

In [23]:
reader = OCRModel() 

In [20]:
import easyocr 
reader2 = easyocr.Reader(['en'], gpu=True) # the reader that will evaluate through the license plate text. 

In [28]:
def Predictions(): 
    PredictionsList = [] 
    for i in TestingDataList:
        x = reader.predict(i)
        char = ""
        if len(x) == 1: 
            char = x[0][1]
            char = char.upper()
            char = char.replace("-","")
            char = char.replace(" ","")
        else:
            for j in x:
                print(j)
                char+= j[1] #j[0]
                char = char.upper()
                char = char.replace("-","")
                char = char.replace(" ","")
        PredictionsList.append(char)
    return PredictionsList

In [29]:
Predictions() 


image 1/1 /home/jik19004/FilesToRun/DINGRESEARCH/5717Proj/LP-Dataset/LP-characters/images/0163.png: 320x640 10 letters, 13.2ms
Speed: 2.5ms preprocess, 13.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
['6']


/home/jik19004/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/jik19004/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/jik19004/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/jik19004/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/jik19004/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/jik19004/anaconda3/lib/p

AttributeError: 'numpy.ndarray' object has no attribute 'upper'